A neural network consist of cnn layer (Kim,2014) and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [3]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [12]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0)
data = data.dropna().reset_index(drop=True)
x = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [14]:
X=[]
for i in x:
  X.append(i.split(" "))

print(X[0])

['خدابا', 'سلام', 'خدمت', 'سون', 'لرنی', 'ها', 'عزیز', 'حالتون', 'باشه', 'آموزش', 'مختصر', 'مفید', 'همراهتون', 'طراحان', 'خیلی', 'ها', 'سرعت', 'راحتی', 'ادیتور', 'Sublime', 'Text', 'میکنن', 'مطمئنا', 'کار', 'کردهباشید', 'اسمش', 'آشنا', 'زبان', 'ها', 'کلمات', 'کلیدی', 'دستور', 'داره', 'کدنویسی', 'زبان', 'اونهارو', 'تایپ', 'تایپ', 'خواهیدکرد', 'تایپ', 'کلمات', 'دستور', 'ها', 'خسته', 'باشه', 'بردن', 'سرعت', 'خودتون', 'میانبر', 'کلمه', 'تایپ', 'آموزش', 'خودمون', 'میانبر', 'همون', 'شورتکات', 'ساده', 'درج', 'متن', 'ایجاد', 'دقت', 'داشتهباشید', 'ایجاد', 'میانبر', 'فرمان', 'command', 'داره', 'فرمان', 'درج', 'متن', 'insertsnippet', 'منو', 'قسمت', 'preferences', 'گزینه', 'ی', 'key', 'bindings', 'کلیک', 'فایل', 'میانبر', 'ها', 'پیشفرض', 'میانبر', 'ها', 'کاربر', 'بشه', 'نگاهی', 'میانبر', 'ها', 'پیشفرض', 'بندازید', 'میبینید', 'فرمت', 'JSON', 'نوشته', 'ایجاد', 'میانبر', 'خودتون', 'فایل', 'میانبر', 'ها', 'کاربر', 'آرایه', 'فرمت', 'JSON', 'نوشتن', 'میانبر', 'ها', 'خودتون', 'بپردازید', 'فایل', 'فرمتی',

In [15]:
len(labels)

78

In [16]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 386


In [17]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [18]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [19]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [20]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [21]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12896, 386) 
test:  (4299, 386) 
val:  (4299, 386) 
y_tain: (12896, 78)


In [22]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [ ]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [23]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [24]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [25]:
from keras.layers import LSTM
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn
    cnn = Conv2D(1, 
                 kernel_size=(5, sentence_maxlen),
                 activation='relu')(embd_sentence)
    cnn =  Lambda(lambda x: K.sum(x, axis=3))(cnn)
    cnn = MaxPooling1D(3)(cnn)
    cnn = Lambda(lambda x: K.sum(x, axis=2))(cnn)
    
    hidden1=Dense(400,activation="relu")(cnn)
    hidden2=Dense(300,activation="relu")(hidden1)
    hidden3=Dense(200,activation="relu")(hidden2)
    hidden4=Dense(150,activation="relu")(hidden3)
    out = Dense(len(labels), activation='sigmoid')(hidden4)
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy',metrics=["accuracy","categorical_accuracy"]) 

    return model

model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())


Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 386)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 386, 300)          67603500  
_________________________________________________________________
permute (Permute)            (None, 300, 386)          0         
_________________________________________________________________
lambda (Lambda)              (None, 300, 386, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 296, 1, 1)         1931      
_________________________________________________________________
lambda_1 (Lambda)            (None, 296, 1)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 98, 1) 

In [26]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) # Model stop training after 50 epoch where validation loss didnt decrease
mc = ModelCheckpoint('bestModelForTest_cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=250,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss

Epoch 1/250
403/403 [==============================] - 69s 168ms/step - loss: 0.2994 - accuracy: 0.0236 - categorical_accuracy: 0.0236 - val_loss: 0.1083 - val_accuracy: 0.0288 - val_categorical_accuracy: 0.0288

Epoch 00001: val_loss improved from inf to 0.10826, saving model to bestModelForTest_cnn_4fc.h5
Epoch 2/250
403/403 [==============================] - 68s 168ms/step - loss: 0.1094 - accuracy: 0.0443 - categorical_accuracy: 0.0443 - val_loss: 0.1054 - val_accuracy: 0.0670 - val_categorical_accuracy: 0.0670

Epoch 00002: val_loss improved from 0.10826 to 0.10540, saving model to bestModelForTest_cnn_4fc.h5
Epoch 3/250
403/403 [==============================] - 68s 168ms/step - loss: 0.1065 - accuracy: 0.0841 - categorical_accuracy: 0.0841 - val_loss: 0.1017 - val_accuracy: 0.1168 - val_categorical_accuracy: 0.1168

Epoch 00003: val_loss improved from 0.10540 to 0.10171, saving model to bestModelForTest_cnn_4fc.h5
Epoch 4/250
403/403 [==============================] - 68s 168ms/

# ***If the model is generated before:***

In [ ]:
model = load_model('CNN_1_no_binary.h5')
# model.save('CNN_1_no_binary.h5')

In [27]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [28]:
print(pred[0])#example

[3.20822001e-04 4.38927873e-06 1.51753426e-04 3.15351571e-07
 1.35601726e-06 4.28532849e-06 2.08553574e-05 2.05227733e-03
 5.13750315e-03 9.07721551e-06 6.86080966e-07 9.78373766e-01
 1.25465095e-02 8.43962407e-05 1.86592340e-04 2.08109617e-04
 3.53664160e-04 5.49089600e-06 4.97367837e-05 1.82523138e-07
 1.14958148e-07 2.61480527e-05 1.07452252e-06 3.67757678e-03
 1.12605983e-06 5.60077242e-05 6.00247131e-06 4.90814447e-04
 5.50224759e-05 1.47663951e-02 8.15478597e-06 3.20272875e-08
 6.58315420e-03 3.72886658e-04 3.65346670e-04 3.78221273e-04
 1.48196518e-02 3.96072865e-04 1.01217120e-05 2.42687776e-07
 5.69272743e-05 5.63025475e-04 7.77104497e-03 4.14699316e-04
 7.38649362e-07 1.44325531e-06 1.10631689e-07 2.24273208e-05
 7.61151314e-04 1.91979834e-05 3.69915366e-03 2.21133232e-04
 5.82216308e-06 1.10737883e-05 1.41402561e-05 2.20924616e-04
 4.87622310e-05 1.05128474e-05 2.27447017e-05 1.73761655e-05
 4.97203973e-06 1.67790972e-07 4.65750694e-04 6.33255504e-10
 1.57923193e-06 3.901387

In [29]:
y_pred=[]
measure = np.mean(pred[0]) + 1.15*np.sqrt(np.var(pred[0]))
for l in pred:
  temp=[]
  for value in l:
    if value >= measure:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [30]:
measure

0.1400433886796236

In [31]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy= 0.2053966038613631
              precision    recall  f1-score   support

           0       0.51      0.38      0.43       114
           1       0.24      0.31      0.27       151
           2       0.35      0.65      0.46       146
           3       0.11      0.14      0.12        14
           4       0.88      0.88      0.88        84
           5       0.00      0.00      0.00        18
           6       0.30      0.59      0.40       165
           7       0.50      0.42      0.45        60
           8       0.29      0.30      0.29        33
           9       0.12      0.34      0.18       100
          10       0.28      0.42      0.34       163
          11       0.87      0.99      0.93        77
          12       0.89      0.97      0.93       136
          13       1.00      0.04      0.08        24
          14       0.12      0.07      0.09        14
          15       0.22      0.39      0.29       155
          16       0.39      0.75      0.52       14

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report,accuracy_score
print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy= 0.21335807050092764
              precision    recall  f1-score   support

           0       0.30      0.40      0.34       123
           1       0.16      0.39      0.23       132
           2       0.39      0.57      0.46       149
           3       0.30      0.32      0.31        19
           4       0.96      0.93      0.94        82
           5       0.00      0.00      0.00        11
           6       0.30      0.59      0.40       152
           7       0.54      0.15      0.23        48
           8       0.18      0.15      0.16        34
           9       0.17      0.15      0.16       123
          10       0.32      0.43      0.37       161
          11       0.85      0.97      0.91        72
          12       0.90      0.97      0.94       142
          13       0.43      0.20      0.27        15
          14       0.00      0.00      0.00        18
          15       0.21      0.47      0.29       148
          16       0.38      0.66      0.48       1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
